In [1]:
%%capture
pip install pandas 
pip install geopandas 
pip install requests 

SyntaxError: invalid syntax (<ipython-input-1-c75d43f3af4c>, line 1)

In [1]:
import rasterio
from rasterio.mask import mask
import geopandas as gpd
import pandas as pd
import numpy as np
from PIL import Image
from tqdm import tqdm

/opt/anaconda3/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [2]:
centr_500 = gpd.read_file("/Volumes/Backup Plus/kursovaya_4/data/vyborka/pk/centroids_500.gpkg", encoding='latin1')
centr_1500 = gpd.read_file("/Volumes/Backup Plus/kursovaya_4/data/vyborka/pk/centroids_1500.gpkg", encoding='latin1')

display(centr_500.head(2))
display(centr_1500.head(2))

,id,left,top,right,bottom,heights_di,dispersion,median,qntl0,qntl25,qntl75,qntl1,geometry
0,217.0,468303.0947,5.881111e+06,468333.0947,5.881141e+06,19.30995,13.102995,6.193358,0.558989,4.189836,8.637076,19.868938,POINT (468303.095 5881111.206)
1,218.0,468303.0947,5.881081e+06,468333.0947,5.881111e+06,19.30995,13.536162,6.193358,0.558989,4.185305,8.698154,19.868938,POINT (468303.095 5881081.206)


,id,left,top,right,bottom,heights_di,dispersion,median,qntl0,qntl25,qntl75,qntl1,geometry
0,217.0,468303.0947,5.881111e+06,468333.0947,5.881141e+06,59.471699,209.606934,20.533821,0.330163,9.177938,31.059659,59.801861,POINT (468303.095 5881111.206)
1,218.0,468303.0947,5.881081e+06,468333.0947,5.881111e+06,59.471699,208.425049,20.447840,0.330163,9.089801,30.915324,59.801861,POINT (468303.095 5881081.206)


In [3]:
centr_1500.rename(columns={'median':'median1500'}, inplace=True)
pk_result = centr_500.merge(centr_1500, on='id', how='right')

In [4]:
pk_result.head(2)

,id,left_x,top_x,right_x,bottom_x,heights_di_x,dispersion_x,median,qntl0_x,qntl25_x,...,right_y,bottom_y,heights_di_y,dispersion_y,median1500,qntl0_y,qntl25_y,qntl75_y,qntl1_y,geometry_y
0,217.0,468303.0947,5.881111e+06,468333.0947,5.881141e+06,19.30995,13.102995,6.193358,0.558989,4.189836,...,468333.0947,5.881141e+06,59.471699,209.606934,20.533821,0.330163,9.177938,31.059659,59.801861,POINT (468303.095 5881111.206)
1,218.0,468303.0947,5.881081e+06,468333.0947,5.881111e+06,19.30995,13.536162,6.193358,0.558989,4.185305,...,468333.0947,5.881111e+06,59.471699,208.425049,20.447840,0.330163,9.089801,30.915324,59.801861,POINT (468303.095 5881081.206)


In [5]:
pk_result["category"] = np.nan
for i in range(len(pk_result)):
    if pk_result['median1500'][i] > pk_result['median'][i] and pk_result['median1500'][i] - pk_result['median'][i]>20:
        pk_result.at[i,'category'] = 1
    elif pk_result['median1500'][i] > pk_result['median'][i] and pk_result['median1500'][i] - pk_result['median'][i]<20:
        pk_result.at[i,'category'] = 2
    elif pk_result['median1500'][i] < pk_result['median'][i] and pk_result['median'][i] - pk_result['median1500'][i]>20:
        pk_result.at[i,'category'] = 3
    elif pk_result['median1500'][i] < pk_result['median'][i] and pk_result['median'][i] - pk_result['median1500'][i]<20:
        pk_result.at[i,'category'] = 4
    else:
        pk_result.at[i,'category'] = 5

print(pk_result)

              id       left_x         top_x      right_x      bottom_x  \
0          217.0  468303.0947  5.881111e+06  468333.0947  5.881141e+06   
1          218.0  468303.0947  5.881081e+06  468333.0947  5.881111e+06   
2          219.0  468303.0947  5.881051e+06  468333.0947  5.881081e+06   
3          220.0  468303.0947  5.881021e+06  468333.0947  5.881051e+06   
4          221.0  468303.0947  5.880991e+06  468333.0947  5.881021e+06   
...          ...          ...           ...          ...           ...   
401547  970154.0  498963.0947  5.879341e+06  498993.0947  5.879371e+06   
401548  970155.0  498963.0947  5.879311e+06  498993.0947  5.879341e+06   
401549  970156.0  498963.0947  5.879281e+06  498993.0947  5.879311e+06   
401550  971103.0  498993.0947  5.879341e+06  499023.0947  5.879371e+06   
401551  971104.0  498993.0947  5.879311e+06  499023.0947  5.879341e+06   

        heights_di_x  dispersion_x    median   qntl0_x  qntl25_x  ...  \
0          19.309950     13.102995  6.

In [6]:
pk_result.head(2)

,id,left_x,top_x,right_x,bottom_x,heights_di_x,dispersion_x,median,qntl0_x,qntl25_x,...,bottom_y,heights_di_y,dispersion_y,median1500,qntl0_y,qntl25_y,qntl75_y,qntl1_y,geometry_y,category
0,217.0,468303.0947,5.881111e+06,468333.0947,5.881141e+06,19.30995,13.102995,6.193358,0.558989,4.189836,...,5.881141e+06,59.471699,209.606934,20.533821,0.330163,9.177938,31.059659,59.801861,POINT (468303.095 5881111.206),2.0
1,218.0,468303.0947,5.881081e+06,468333.0947,5.881111e+06,19.30995,13.536162,6.193358,0.558989,4.185305,...,5.881111e+06,59.471699,208.425049,20.447840,0.330163,9.089801,30.915324,59.801861,POINT (468303.095 5881081.206),2.0


In [17]:
pk_result.to_csv("/Users/panssies/Desktop/pk_result.csv")